In [20]:
import json
import time
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.output_parsers import PydanticOutputParser
from IPython.display import display, Markdown, JSON

In [2]:
load_dotenv(override=True) # loading API key from environment

True

## Read in data with annual curriculum plan

In [5]:
sheet_url = "https://docs.google.com/spreadsheets/d/1Fpwrp1v1H10e02y5K1JqVBUu9yH5kLjlTDk0w2ZNfpc/edit#gid=0"
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
annual_plan_df = pd.read_csv(url_1)
annual_plan_df.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
annual_plan_df

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,"Getting to Know One Another, Class Community",All About Me: Personal Narratives,Friendship Math: Counting and Sorting,Our Classroom Community,Exploring Our School Environment and Local Ec...
1,September,On the Map/where we’re from,My Place on the Map: Descriptive Writing,Mapping Our Neighborhood,My Hometown and its history,Geography and Landforms Around Us
2,October,Fall/Halloween history/Dia de los Muertos,Spooky Stories: Creative Writing,Pumpkin Math: Estimation and Graphs,Cultural Celebrations,Investigating Seasonal Changes and Plants
3,November,Gratitude; Native Americans,Thankful Thoughts: Gratitude Journals,Native American Patterns and Art,Native American Heritage,Animal science: Investigating Local Food Chains
4,December,Winter Solstice and winter holidays,Holiday Traditions Around the World,Winter Math: Snowflake Symmetry,Winter Holidays Worldwide,Exploring Winter Animal Adaptations
5,January,MLK Calendar (New Year),Dreams and Goals: Writing Prompts,Calendar Math: Days and Months,MLK and Civil Rights Movement,Climate Zones and Seasons
6,February,"Black History, Love/Valentine's Day",Celebrating African American Heroes,Math of Love: Patterns and Geometry,African American Achievements,"The human body, heart health and exercise"
7,March,"SpringEaster, Ramadan",Springtime Poetry and Descriptions,Geometry in Islamic Art,"Cultural Spring Celebrations (Easter, Ramadan)",Life Cycles of Plants and Animals (frogs and b...
8,April,Space; Passover,Cosmic Adventures: Space Stories,Planet Math: Solar System,Passover Origins Ancient Astronomy,Exploring the Solar System and Celestial Bodies
9,May,Ocean Life,Ocean Explorers: Descriptive Writing,Dive into Math: Ocean Measurements,Coastal Ecosystems,Marine Life and Habitats


## Instantiate the model

In [4]:
model_name = "text-davinci-003"
temperature = 0.4
model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)

# Create Prompt Templates

In [7]:
class ActivityIdea(BaseModel):
    name: str
    description: str

class ActivityIdeasOutput(BaseModel):
    ideas: List[ActivityIdea]
# Set up a parser + inject instructions into the prompt template.
activity_ideas_parser = PydanticOutputParser(pydantic_object=ActivityIdeasOutput)

In [8]:
activity_idea_prompt = PromptTemplate(
    template="""Answer the user query.
    {format_instructions}
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    I want you to generate four activity ideas for the month of {month}. Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I have a theme for the month, which is {theme}.
    The academic subject is {subject} and the topic is {topic}.

    Here are some additional guidelines:
    - Try to be creative. Do not use the idea of a scavenger hunt, because it has been overdone.
    - Try to work in phonics work and literacy development into the language art activities.
    - Each activity idea should be able to be accomplished in 30 mins-3 hours.
    - Each activity should relate directly to skills and concepts from the academic subject of {subject}.
    - I only need you to give me the activity names and a brief description of each idea.
    - The description should only be sentence or two and should not simply repeat the theme or the topic.
    - Keep in mind these are second-graders who are not ready for advanced concepts like decimals.
    """
    ,
    input_variables=["month", "theme", "subject", "topic"],
    partial_variables={"format_instructions": activity_ideas_parser.get_format_instructions()}
)

In [9]:
lesson_plan_prompt = PromptTemplate(
    template="""Answer the user query.
    You are a curriculum development expert and i am a teacher of second graders at a micro-school, with seven students of varying abilities.
    I want you to generate a lesson plan.
    The name of the lesson is {activity_name}, and a short description is: {activity_description}
    ...
    The output should be in valid Markdown format. 
    Ensure that there's a consistent use of spaces for indentation before each heading to make sure they are recognized and displayed properly. 

    The Markdown output should have the following structure:
    \n## {activity_name}
    \n### {activity_description}
    \n### Time Estimate
    \n### List of Materials
    \n### Procedure
    \n### Additional Notes
    \n### Differentiation strategies
    ...

    Here are the things you should keep in mind when generating the lesson plan:
    - The microschool  stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Ensure that the lesson uses and develops skills relevant to the academic subject: {subject}
    - Try to make the best estimate of the total amount of time the activity will take. Be generous because these are young kids and they take a while!
    - The additional notes should include examples for the teacher, to help bring the procedure to life, as needed and applicable. 
    - The differentiation strategies should provide modifications for students who need more support, for students who need more challenge, and for neurodivergent students.
    """
    ,
    input_variables=["activity_name", "activity_description", "subject"]
)

In [10]:
def create_activity_ideas(activity_idea_prompt, parser, month, theme, subject, topic, max_retries=5):
        res ={}
        _activity_idea_input = activity_idea_prompt.format_prompt(
            month=month, 
            theme=theme, 
            subject=subject, 
            topic=topic 
        )
        for attempt in range(max_retries + 1):
            try:
                activity_ideas_output = model(_activity_idea_input.to_string())
                activity_ideas_parsed_output = parser.parse(activity_ideas_output)
                break
            except Exception as e:
                if attempt < max_retries:
                    print(f"Attempt {attempt + 1} failed. Retrying...")
                    time.sleep(2)  # Wait before retrying
                else:
                    print(f"API call failed after {max_retries} attempts. Skipping.")
        return activity_ideas_parsed_output

In [11]:
def create_lesson_plan(idea_name, idea_description, lesson_plan_prompt, subject):
        _lesson_plan_input = lesson_plan_prompt.format_prompt(
            activity_name = idea_name, 
            activity_description = idea_description,
            subject=subject
        )
        lesson_plan_output = model(_lesson_plan_input.to_string())
        lesson_plan_output_clean = lesson_plan_output.replace("    ", "")
        # display(Markdown(lesson_plan_output_clean))
        return lesson_plan_output_clean

In [17]:
def wrapper(df):
    res = {}
    for idx, row in df.iterrows():
        month = row["month"]
        theme = row["theme_to_explore"]
        print(f"# {month}")
        mkdwn_str = f"\n# {month}\n## Theme: {theme}"
        res[month] = dict()
        subjects = df.columns[2:].values
        for subject in subjects:
            pretty_subject = subject.replace("_", " ").title()
            mkdwn_str += f"\n\n# {pretty_subject}"
            print(f"- {pretty_subject}")
            topic = row[subject]
            activity_ideas_parsed = create_activity_ideas(
                activity_idea_prompt, 
                activity_ideas_parser, 
                month,
                theme,
                subject, 
                topic
            )
            res[month][subject] = activity_ideas_parsed.dict()
            for idea in res[month][subject]["ideas"]:
                idea_name = idea["name"]
                idea_description = idea["description"]
                lesson_plan = create_lesson_plan(
                    idea_name, 
                    idea_description, 
                    lesson_plan_prompt,
                    subject
                )
                idea['lesson_plan'] = lesson_plan
                mkdwn_str += f"\n\n {lesson_plan}"
                
        # Write the output to a Markdown file
        output_file_path = f'../output/{idx + 1} {month} Lesson Plans.md'
        with open(output_file_path, "w") as output_file:
            output_file.write(mkdwn_str)

        print(f"Markdown output written to '{output_file_path}'.")
        print("*" * 40, "\n")
    
    import json

    # Write dictionary to a JSON file
    full_result_output_path = "../output/lesson_plan.json"
    with open(full_result_output_path , "w") as json_file:
        json.dump(res, json_file)
    print(f"JSON output written to '{full_result_output_path }'.")
    return res

In [18]:
# aug_sep = wrapper(df.iloc[:2])
# oct_nov_dec = df.iloc[3:5]
# jan_feb = wrapper(df.iloc[5:7)
# mar_april_may = wrapper(df.iloc[7:])

In [21]:
lesson_plans = wrapper(annual_plan_df)

# August
- Language Arts


KeyboardInterrupt: 

In [ ]:
display(JSON(lesson_plans))